## **Ejercicio en clase 1 - ETL**

Daniela Gómez Ayalde

#### **Librerías**

Se importan las librerías necesarias para ejecutar el código correctamente.

In [1]:
import yaml
import psycopg2
import pandas as pd
from psycopg2 import sql
from sqlalchemy import create_engine, text

#### **Lectura de CSV**

Se leen los datos en formato *.csv* utilizando la librería Pandas.

In [3]:
df = pd.read_csv("candidates.csv", delimiter=";")
df

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


#### **Cargar archivo de configuración**

Se define una función *load_config* que carga la configuración desde un archivo YAML y devuelve su contenido en forma de un diccionario de Python. 

In [4]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

#### **Conexión a la base de datos**

Se establece la conexión a una base de datos PostgreSQL utilizando la configuración almacenada en el archivo YAML.

In [ ]:
config = load_config()
db_config = config["database"]

# Cargar credenciales
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# Conexión a la DB
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

#### **Crear base de datos**

Se intenta crear una nueva base de datos en PostgreSQL utilizando la conexión establecida previamente. 

In [6]:
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close()

Base de datos 'ejercicio1_db' creada exitosamente.


#### **Crear tabla que contendrá los datos crudos**

Se utiliza SQLAlchemy para conectarse a la base de datos PostgreSQL y crear una tabla llamada *'raw_data'* si aún no existe.

In [7]:
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS raw_data (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(100),
            application_date DATE,
            country VARCHAR(100),
            yoe INT,
            seniority VARCHAR(100),
            technology VARCHAR(100),
            code_challenge_score INT,
            technical_interview_score INT
        );
    """))
    conn.commit()  # Asegúrate de confirmar los cambios
    print("Tabla 'raw_data' creada exitosamente en PostgreSQL.")

Tabla 'raw_data' creada exitosamente en PostgreSQL.


#### **Cambiar nombre de columnas en dataframe**

Se cambia el nombre de las columnas del dataframe para que coincidan con los nombres de las columnas de la tabla creada anteriormente.

In [8]:
df.columns = ['first_name', 'last_name', 'email', 'application_date', 'country', 'yoe', 'seniority', 'technology', 'code_challenge_score', 'technical_interview_score']
df

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


#### **Cargar datos a la tabla**

Se inserta el DataFrame en la tabla *'raw_data'* de la base de datos utilizando SQLAlchemy.

In [9]:
with engine.connect() as conn:
    df.to_sql("raw_data", con=engine, if_exists="append", index=False)

print("Raw data stored successfully in table 'raw_data'.")

Raw data stored successfully in table 'raw_data'.


#### **Leer datos de la tabla**

Este código consulta y muestra los primeros 50 registros de la tabla *'raw_data'* en la base de datos PostgreSQL.

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM raw_data;"))
    rows = result.fetchall()

# Mostrar los primeros 50 resultados
print("Datos en 'raw_data':")
for row in rows[:50]:
    print(row)

Datos en 'raw_data':
(1, 'Bernadette', 'Langworth', 'leonard91@yahoo.com', datetime.date(2021, 2, 26), 'Norway', 2, 'Intern', 'Data Engineer', 3, 3)
(2, 'Camryn', 'Reynolds', 'zelda56@hotmail.com', datetime.date(2021, 9, 9), 'Panama', 10, 'Intern', 'Data Engineer', 2, 10)
(3, 'Larue', 'Spinka', 'okey_schultz41@gmail.com', datetime.date(2020, 4, 14), 'Belarus', 4, 'Mid-Level', 'Client Success', 10, 9)
(4, 'Arch', 'Spinka', 'elvera_kulas@yahoo.com', datetime.date(2020, 10, 1), 'Eritrea', 25, 'Trainee', 'QA Manual', 7, 1)
(5, 'Larue', 'Altenwerth', 'minnie.gislason@gmail.com', datetime.date(2020, 5, 20), 'Myanmar', 13, 'Mid-Level', 'Social Media Community Management', 9, 7)
(6, 'Alec', 'Abbott', 'juanita_hansen@gmail.com', datetime.date(2019, 8, 17), 'Zimbabwe', 8, 'Junior', 'Adobe Experience Manager', 2, 9)
(7, 'Allison', 'Jacobs', 'alba_rolfson27@yahoo.com', datetime.date(2018, 5, 18), 'Wallis and Futuna', 19, 'Trainee', 'Sales', 2, 9)
(8, 'Nya', 'Skiles', 'madisen.zulauf@gmail.com', da

#### **Convertir a dataframe los datos de la tabla**

Se consulta todos los registros de la tabla *'raw_data'* en la base de datos y los carga directamente en un DataFrame de pandas (df_db).

In [11]:
with engine.connect() as conn:
    df_db = pd.read_sql("SELECT * FROM raw_data", conn)

df_db

,id,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score
0,1,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,2,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,3,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,4,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,5,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,49997,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,49998,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,49999,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


#### **Realizar transformaciones**

Este código crea una copia del DataFrame original y agrega una nueva columna llamada "hired" que indica si un candidato fue contratado o no, basado en sus puntajes.

In [ ]:
df_transformed = df_db.copy()
df_transformed["hired"] = df_transformed.apply(
    lambda x: "Hired" if x["code_challenge_score"] >= 7 and x["technical_interview_score"] >= 7 else "Not Hired", axis=1)

# Mostrar las primeras 5 filas del Dataframe transformado
df_transformed.head()

,id,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired
0,1,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,Not Hired
1,2,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,Not Hired
2,3,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9,Hired
3,4,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1,Not Hired
4,5,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7,Hired


#### **Crear tabla que contendrá los datos transformados**

Se intenta crear una tabla llamada *'transformed_data'* en la base de datos PostgreSQL para almacenar los datos transformados.

In [13]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS transformed_data (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(100),
            application_date DATE,
            country VARCHAR(100),
            yoe INT,
            seniority VARCHAR(100),
            technology VARCHAR(100),
            code_challenge_score INT,
            technical_interview_score INT,
            hired VARCHAR(20)
        );
    """))
    conn.commit()  # Asegúrate de confirmar los cambios
    print("Tabla 'transformed_data' creada exitosamente en PostgreSQL.")

Tabla 'transformed_data' creada exitosamente en PostgreSQL.


#### **Cargar los datos transformados a la tabla**

Se insertan los datos del DataFrame *df_transformed* en la tabla *'transformed_data'* de la base de datos PostgreSQL utilizando SQLAlchemy y Pandas.

In [14]:
with engine.connect() as conn:
    df_transformed.to_sql("transformed_data", con=engine, if_exists="append", index=False)

print("Datos almacenados correctamente en la tabla 'transformed_data'.")

Datos almacenados correctamente en la tabla 'transformed_data'.


#### **Leer datos de la tabla**

Este código verifica si la tabla *'transformed_data'* se ha creado correctamente y muestra los primeros 50 registros almacenados.

In [18]:
# Leer datos de la tabla para verificar que haya sido creada correctamente
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM transformed_data;"))
    rows = result.fetchall()

# Mostrar los primeros 50 resultados
print("Datos en 'transformed_data':")
for row in rows[:50]:
    print(row)

Datos en 'transformed_data':
(1, 'Bernadette', 'Langworth', 'leonard91@yahoo.com', datetime.date(2021, 2, 26), 'Norway', 2, 'Intern', 'Data Engineer', 3, 3, 'Not Hired')
(2, 'Camryn', 'Reynolds', 'zelda56@hotmail.com', datetime.date(2021, 9, 9), 'Panama', 10, 'Intern', 'Data Engineer', 2, 10, 'Not Hired')
(3, 'Larue', 'Spinka', 'okey_schultz41@gmail.com', datetime.date(2020, 4, 14), 'Belarus', 4, 'Mid-Level', 'Client Success', 10, 9, 'Hired')
(4, 'Arch', 'Spinka', 'elvera_kulas@yahoo.com', datetime.date(2020, 10, 1), 'Eritrea', 25, 'Trainee', 'QA Manual', 7, 1, 'Not Hired')
(5, 'Larue', 'Altenwerth', 'minnie.gislason@gmail.com', datetime.date(2020, 5, 20), 'Myanmar', 13, 'Mid-Level', 'Social Media Community Management', 9, 7, 'Hired')
(6, 'Alec', 'Abbott', 'juanita_hansen@gmail.com', datetime.date(2019, 8, 17), 'Zimbabwe', 8, 'Junior', 'Adobe Experience Manager', 2, 9, 'Not Hired')
(7, 'Allison', 'Jacobs', 'alba_rolfson27@yahoo.com', datetime.date(2018, 5, 18), 'Wallis and Futuna', 19,